In [1]:
!pip install autots
# Read in data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import time
import datetime
from sklearn.model_selection import train_test_split
from autots import AutoTS

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Import data

In [3]:
#df

In [4]:
# Define years and months to load data from both 2023 and 2024
years = [2023, 2024]
months = range(1, 13)

# Load datasets dynamically for both years
data_files = [
    f"ig_data-{year}/ig_data_{str(month).zfill(2)}-{year}_u.csv"
    for year in years
    for month in months
]

datasets = [pd.read_csv(file) for file in data_files]

# Add month and year columns
for i, (year, month) in enumerate([(y, m) for y in years for m in months]):
    datasets[i]["month"] = month
    datasets[i]["year"] = year

In [5]:
# Concatenate data
data = pd.concat(datasets)
data

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year
0,2023-01-01,26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...,20558,0,0,0,0,0,0,1,2023
1,2023-01-01,31832168975075c19bfca97103d24b2525b75235b2e61a...,5256,0,0,0,0,0,0,1,2023
2,2023-01-01,065c756ab25ca5147325477d859c320577aa171e55d99b...,337483,1,149,145,4,4101,4511,1,2023
3,2023-01-01,ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...,62556,0,0,0,0,0,0,1,2023
4,2023-01-01,27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...,359455,1,3997,3509,488,57388,63127,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
547988,2024-12-31,8c1ac76cb45374c9710165d154d153ddf48bc30c253c8e...,5905,0,0,0,0,0,0,12,2024
547989,2024-12-31,834715aec99b1e9be4feccadeea4712f1cde42d649664d...,211245,1,345,337,8,10157,11172,12,2024
547990,2024-12-31,94528532ec6bee015266fdaeb9cd570a6fbf6e3a2dfbc2...,1660,0,0,0,0,0,0,12,2024
547991,2024-12-31,0597c04c318d668379c1d549c31931c56838f6865ae7f3...,10084,1,28,28,0,2914,3205,12,2024


In [6]:
# find in the column followers the rows that have the value as "undefined"
data[data["followers"] == "undefined"]
# replace the "undefined" in follwoers with nan
data["followers_num"] = data["followers"].replace("undefined", np.nan)
# convert the followers_num to integer
data["followers_num"] = data["followers_num"].astype(float).astype(pd.Int64Dtype())
# Ensure followers_num is numeric and clean
data["followers_num"] = pd.to_numeric(data["followers_num"], errors="coerce")

# Step 4: Replace zeros with NaN (to avoid filling with zero)
data["followers_num"].replace(0, np.nan, inplace=True)

# Step 5: Fill NaN values using the last non-null value, making sure it isn't zero
data["followers_num"] = data["followers_num"].fillna(method="ffill")

# Step 6: If still NaN (if there were no valid previous non-null value), you can set to zero or some other value
data["followers_num"].fillna(0, inplace=True)

# Step 7: Ensure the column type is integer, but with nullable integer type to handle missing values
data["followers_num"] = data["followers_num"].astype(pd.Int64Dtype())

# Verify the changes
print(data.head())


         date                                         profile_id followers  \
0  2023-01-01  26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...     20558   
1  2023-01-01  31832168975075c19bfca97103d24b2525b75235b2e61a...      5256   
2  2023-01-01  065c756ab25ca5147325477d859c320577aa171e55d99b...    337483   
3  2023-01-01  ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...     62556   
4  2023-01-01  27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...    359455   

   posts  engagement  likes  comments  reach  impressions  month  year  \
0      0           0      0         0      0            0      1  2023   
1      0           0      0         0      0            0      1  2023   
2      1         149    145         4   4101         4511      1  2023   
3      0           0      0         0      0            0      1  2023   
4      1        3997   3509       488  57388        63127      1  2023   

   followers_num  
0          20558  
1           5256  
2         337483  
3         

In [7]:
data.isnull().sum()

date             0
profile_id       0
followers        0
posts            0
engagement       0
likes            0
comments         0
reach            0
impressions      0
month            0
year             0
followers_num    0
dtype: int64

In [8]:
data[data["followers_num"] == "undefined"]

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year,followers_num


In [9]:
profile_counts = data["profile_id"].value_counts()
# # get the ids of the accounts that have more than 365 records
# more_than_299_ids = profile_counts[profile_counts >= 300].index
# # filter the data to include only the accounts that have more than 365 records
# more_than_299_data = data[data["profile_id"].isin(more_than_299_ids)]
# get the ids of the accounts that have more than 365 records
more_than_365_ids = profile_counts[profile_counts > 365].index
# filter the data to include only the accounts that have more than 365 records
more_than_365_data = data[data["profile_id"].isin(more_than_365_ids)]
# get the ids of the accounts that have 300 to 365 records
between_365_300_ids = profile_counts[
    (profile_counts <= 365) & (profile_counts >= 300)
].index
# filter the data to include only the accounts that have 300 to 365 records
between_365_300_data = data[data["profile_id"].isin(between_365_300_ids)]
# get the ids of the accounts that have 90 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 90 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [12]:
# Export more than 365 data to csv
between_365_300_data.to_csv("between_365_300_data.csv", index=False)

# Export more than 365 data to csv
between_299_90_data.to_csv("between_299_90_data.csv", index=False)